In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd

In [23]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 10
DEVICE = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Qurasi.id/Dataset/data_berlabel.csv')
df = df.drop(columns='Unnamed: 0')

In [5]:
df = df.dropna()

In [6]:
df.to_csv('data_berlabel.csv', index=False)

In [7]:
from torchtext.data import Field, LabelField

# Defining the feature processing
TEXT = Field()  # default splits on whitespace

# Defining the label processing
LABEL = LabelField(dtype=torch.long)


In [8]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.data.TabularDataset(
    path='/content/data_berlabel.csv', format='csv',
    skip_header=True, fields=fields)

In [9]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 10481
Num Test: 2620


In [10]:
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')

Num Train: 8909
Num Validation: 1572


In [11]:
print(vars(train_data.examples[0]))

{'TEXT_COLUMN_NAME': ['prabowo', 'mesti', 'tanggung', 'bangun', 'sekolah', 'bakar'], 'LABEL_COLUMN_NAME': '0'}


In [12]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 13000
Number of classes: 2


In [13]:
print(TEXT.vocab.freqs.most_common(20))

[('yang', 2012), ('orang', 769), ('jokowi', 566), ('presiden', 560), ('islam', 509), ('agama', 492), ('indonesia', 458), ('pilih', 339), ('cebong', 310), ('cina', 287), ('asing', 266), ('gubernur', 254), ('rakyat', 245), ('kepala', 237), ('rezim', 233), ('kristen', 232), ('ahok', 228), ('negara', 226), ('komunis', 225), ('daerah', 223)]


In [14]:
print(TEXT.vocab.itos[:10]) # itos = integer-to-string

['<unk>', '<pad>', 'yang', 'orang', 'jokowi', 'presiden', 'islam', 'agama', 'indonesia', 'pilih']


In [15]:
print(TEXT.vocab.stoi['the']) # stoi = string-to-integer

258


In [16]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


In [17]:
train_loader, valid_loader, test_loader = \
    torchtext.data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=DEVICE
    )

In [18]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([26, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([2, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([1, 128])
Target vector size: torch.Size([128])


In [24]:
class LSTM(torch.nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)

        self.fc = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, text):
        # text dim: [sentence length, batch size]

        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]

        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]

        output = self.fc(hidden)
        return output

In [25]:
torch.manual_seed(RANDOM_SEED)
model = LSTM(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [26]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [27]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):

        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()

        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()

        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')

    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/010 | Batch 000/070 | Loss: 0.6886
Epoch: 001/010 | Batch 050/070 | Loss: 0.4542
training accuracy: 78.80%
valid accuracy: 79.13%
Time elapsed: 0.34 min
Epoch: 002/010 | Batch 000/070 | Loss: 0.5491
Epoch: 002/010 | Batch 050/070 | Loss: 0.5758
training accuracy: 78.82%
valid accuracy: 79.26%
Time elapsed: 0.69 min
Epoch: 003/010 | Batch 000/070 | Loss: 0.4985
Epoch: 003/010 | Batch 050/070 | Loss: 0.6031
training accuracy: 78.79%
valid accuracy: 77.61%
Time elapsed: 1.02 min
Epoch: 004/010 | Batch 000/070 | Loss: 0.4514
Epoch: 004/010 | Batch 050/070 | Loss: 0.2593
training accuracy: 98.10%
valid accuracy: 98.09%
Time elapsed: 1.35 min
Epoch: 005/010 | Batch 000/070 | Loss: 0.1236
Epoch: 005/010 | Batch 050/070 | Loss: 0.0094
training accuracy: 99.32%
valid accuracy: 98.54%
Time elapsed: 1.68 min
Epoch: 006/010 | Batch 000/070 | Loss: 0.0298
Epoch: 006/010 | Batch 050/070 | Loss: 0.0270
training accuracy: 99.71%
valid accuracy: 99.24%
Time elapsed: 2.06 min
Epoch: 007/010 |

In [31]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [48]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = word_tokenize(sentence)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    predicted_label = torch.argmax(prediction, dim=1).item()
    return predicted_label

print('Predicted label:')
predicted_label = predict_sentiment(model, "mukalu goblog tolol idiot")
print(predicted_label)


Predicted label:
1


In [49]:
predicted_label = predict_sentiment(model, "dasar kamu baik hati dan tidak sombong")
print(predicted_label)

0


In [47]:
torch.save(model.state_dict(), 'modeltext_qurasi.pth')